In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

In [51]:
import datetime as dt

In [67]:
last_week=(dt.datetime.now() - dt.timedelta(days=7)).strftime("%Y-%m-%d")

In [2]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results

In [192]:
# Ask for a date in the past 7 days and the app below will base on the date entered"
end_date=input("Which date of the last 7 days would you like to end? YYYY-MM-DD")

Which date of the last 7 days would you like to end? YYYY-MM-DD2019-04-14


In [193]:
# Display time frame
print(f'With Standard Account, you have chosen the time frame from {last_week} to {end_date}.')
print("Update to Premium Account to see more.")

With Standard Account, you have chosen the time frame from 2019-04-08 to 2019-04-14.
Update to Premium Account to see more.


In [194]:
premium_search_args = load_credentials("twitter_keys.yaml",
                                       yaml_key="search_tweets_premium",
                                       env_overwrite=False)
print(premium_search_args)

Grabbing bearer token from OAUTH


{'bearer_token': 'AAAAAAAAAAAAAAAAAAAAAFym9wAAAAAAX5fT5z2UOozB%2BBWMU1Y5jpw2rs8%3DWGoz5Y2GyMNPs4y8nySXjm7ANrhCoOWMtYvchfSoNC5WDsBKpU', 'endpoint': 'https://api.twitter.com/1.1/tweets/search/fullarchive/dev.json', 'extra_headers_dict': None}


In [207]:
premium_search_args.get('bearer_token')
headers={'Authorization': 'Bearer ' + premium_search_args.get('bearer_token')}
keyword1="StockMarket"
d=end_date
url1='https://api.twitter.com/1.1/search/tweets.json?q='+keyword+'&until='+d+"lang=en"
r1 = requests.get(url,headers=headers).json()

dict_keys(['statuses', 'search_metadata'])

In [206]:
len(r['statuses'])


9

In [181]:
r['statuses'][0]

{'created_at': 'Mon Apr 08 23:59:59 +0000 2019',
 'id': 1115403951627575297,
 'id_str': '1115403951627575297',
 'text': "Bombshell report links Trump's Azerbaijan hotel to Iran's Revolutionary Guard https://t.co/DdmtMQ3ZLi",
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/DdmtMQ3ZLi',
    'expanded_url': 'https://www.rawstory.com/2019/04/bombshell-report-links-trumps-azerbaijan-hotel-irans-revolutionary-guard/#.XKvgeq8HIn8.twitter',
    'display_url': 'rawstory.com/2019/04/bombsh…',
    'indices': [78, 101]}]},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 3098361703,
  'id_str': '3098361703',
  'name': 'Richard Forrester',
  'scree

In [4]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in cities_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['City']
    state = row['State']

    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
cities_pd.head()

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,,,
1,Los Angeles,California,34.0522,-118.244,,,
2,Chicago,Illinois,41.8781,-87.6298,,,
3,Houston,Texas,29.7604,-95.3698,,,
4,Philadelphia,Pennsylvania,39.9526,-75.1652,,,


In [5]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "airport",
    "keyword": "international airport",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in cities_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        cities_pd.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [6]:
# Save Data to csv
cities_pd.to_csv("Airport_Output.csv")

# Visualize to confirm airport data appears
cities_pd.head(10)

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.2
1,Los Angeles,California,34.0522,-118.244,Los Angeles International Airport,"1 World Way, Los Angeles",3.6
2,Chicago,Illinois,41.8781,-87.6298,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",3.6
3,Houston,Texas,29.7604,-95.3698,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",3.7
4,Philadelphia,Pennsylvania,39.9526,-75.1652,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.3
5,Phoenix,Arizona,33.4484,-112.074,Phoenix Sky Harbor International Airport,"3400 E Sky Harbor Blvd, Phoenix",3.9
6,San Antonio,Texas,29.4241,-98.4936,San Antonio International Airport,"9800 Airport Blvd, San Antonio",4
7,San Diego,California,32.7157,-117.161,San Diego International Airport,"3225 N Harbor Dr, San Diego",3.9
8,Dallas,Texas,32.7767,-96.797,Dallas/Fort Worth International Airport,"2400 Aviation Dr, DFW Airport",3.8
9,San Jose,California,37.3382,-121.886,San Francisco International Airport,San Francisco,4.1
